In [2]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window
from sklearn import preprocessing # https://github.com/Snowflake-Labs/snowpark-python-demos/tree/main/sp4py_utilities
from snowflake.snowpark.functions import col

import getpass
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
accountname = 'ex39201.ap-southeast-1'
username = getpass.getpass()    # SNOWFLAKE-USERNAME
password = getpass.getpass()    # SNOWFLAKE-PASSWORD

In [4]:
connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "ACCOUNTADMIN",
    "database": "FROSTBYTE_TASTY_BYTES",

    "warehouse": "HOL_WH"
}

session = Session.builder.configs(connection_parameters).create()

In [5]:
session.use_schema("ANALYTICS")
X_final_scaled=session.sql('Select * from "Sales_Forecast_Training_Data"').to_pandas()

X_final_scaled.rename(columns={"Profit": "Revenue"},inplace=True)

In [ ]:
#CAPPING CODE
def winsorise(df, variable, upper_limit, lower_limit):
    return np.where(df[variable] > upper_limit, upper_limit, 
                    np.where(df[variable] < lower_limit, lower_limit, df[variable]))

#X_final_scaled['Revenue']= winsorise(X_final_scaled, 'Revenue', X_final_scaled['Revenue'].quantile(0.85), X_final_scaled['Revenue'].quantile(0))
#X_final_scaled['SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE']= winsorise(X_final_scaled, 'SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE', X_final_scaled['SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE'].quantile(0.85), X_final_scaled['SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE'].quantile(0))
#X_final_scaled['SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE']= winsorise(X_final_scaled, 'SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE', X_final_scaled['SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE'].quantile(0.8), X_final_scaled['SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE'].quantile(0.5))

In [ ]:
#TRIMMING CODE
def trim_outliers(dataframe, column, lower_percentile=0.01, upper_percentile=0.99):
    lower_bound = dataframe[column].quantile(lower_percentile)
    upper_bound = dataframe[column].quantile(upper_percentile)
    return dataframe[(dataframe[column] >= lower_bound) & (dataframe[column] <= upper_bound)]

#X_final_scaled = trim_outliers(X_final_scaled, 'Revenue')

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math

# Split the dataset into features (X) and target (y)
X = X_final_scaled.drop("Revenue",axis=1)
y = X_final_scaled["Revenue"]

# Split the dataset into training and testing datasets
X_training, X_holdout, y_training, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2, random_state=42)

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(objective="reg:squarederror", learning_rate=0.01523, max_depth=9, colsample_bytree=0.578, n_estimators=641, subsample=0.854)
xgb.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)])

print('Train MSE is: ', mean_squared_error(xgb.predict(X_train), y_train))
print('Test MSE is: ', mean_squared_error(xgb.predict(X_test), y_test))
print()
print('Train RMSE is: ',  math.sqrt(mean_squared_error(xgb.predict(X_train), y_train)))
print('Test RMSE is: ', math.sqrt(mean_squared_error(xgb.predict(X_test), y_test)))
print()
print('Train MAE is: ', mean_absolute_error(xgb.predict(X_train), y_train))
print('Test MAE is: ', mean_absolute_error(xgb.predict(X_test), y_test))
print()
print('Train R2 is: ', r2_score(xgb.predict(X_train), y_train))
print('Test R2 is: ', r2_score(xgb.predict(X_test), y_test))

In [9]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

print('Train MSE is: ', mean_squared_error(xgb.predict(X_train), y_train))
print('Test MSE is: ', mean_squared_error(xgb.predict(X_test), y_test))
print()
print('Train RMSE is: ',  math.sqrt(mean_squared_error(xgb.predict(X_train), y_train)))
print('Test RMSE is: ', math.sqrt(mean_squared_error(xgb.predict(X_test), y_test)))
print()
print('Train MAE is: ', mean_absolute_error(xgb.predict(X_train), y_train))
print('Test MAE is: ', mean_absolute_error(xgb.predict(X_test), y_test))
print()
print('Train R2 is: ', r2_score(xgb.predict(X_train), y_train))
print('Test R2 is: ', r2_score(xgb.predict(X_test), y_test))

c:\Users\Jammie99\anaconda3\envs\pysnowpark\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
c:\Users\Jammie99\anaconda3\envs\pysnowpark\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Train MSE is:  3037134.447084011
Test MSE is:  3303912.8277955465

Train RMSE is:  1742.737630018934
Test RMSE is:  1817.6668638107333

Train MAE is:  1174.180126041523
Test MAE is:  1206.6794287798798

Train R2 is:  0.905077538942739
Test R2 is:  0.896930194449483


In [7]:
from sklearn. ensemble import RandomForestRegressor

#Build Random Forest Model and Evaluate both training and testing accuracy
rf = RandomForestRegressor()
rf.fit(X_train,y_train)

print('Train MSE is: ', mean_squared_error(rf.predict(X_train), y_train))
print('Test MSE is: ', mean_squared_error(rf.predict(X_test), y_test))
print()
print('Train RMSE is: ',  math.sqrt(mean_squared_error(rf.predict(X_train), y_train)))
print('Test RMSE is: ', math.sqrt(mean_squared_error(rf.predict(X_test), y_test)))
print()
print('Train MAE is: ', mean_absolute_error(rf.predict(X_train), y_train))
print('Test MAE is: ', mean_absolute_error(rf.predict(X_test), y_test))
print()
print('Train R2 is: ', r2_score(rf.predict(X_train), y_train))
print('Test R2 is: ', r2_score(rf.predict(X_test), y_test))

Train MSE is:  504353.78454797977
Test MSE is:  3572740.5598396068

Train RMSE is:  710.1786990243933
Test RMSE is:  1890.1694526786764

Train MAE is:  465.0162163319141
Test MAE is:  1246.9255194546001

Train R2 is:  0.9848667109612783
Test R2 is:  0.8896925026934712


In [8]:
from sklearn. ensemble import BaggingRegressor

rf = BaggingRegressor(base_estimator=None, n_estimators=10, random_state=0)
rf.fit(X_train,y_train)

print('Train MSE is: ', mean_squared_error(rf.predict(X_train), y_train))
print('Test MSE is: ', mean_squared_error(rf.predict(X_test), y_test))
print()
print('Train RMSE is: ',  math.sqrt(mean_squared_error(rf.predict(X_train), y_train)))
print('Test RMSE is: ', math.sqrt(mean_squared_error(rf.predict(X_test), y_test)))
print()
print('Train MAE is: ', mean_absolute_error(rf.predict(X_train), y_train))
print('Test MAE is: ', mean_absolute_error(rf.predict(X_test), y_test))
print()
print('Train R2 is: ', r2_score(rf.predict(X_train), y_train))
print('Test R2 is: ', r2_score(rf.predict(X_test), y_test))

Train MSE is:  697277.3357231444
Test MSE is:  3883356.9315399034

Train RMSE is:  835.0313381683015
Test RMSE is:  1970.6234880209622

Train MAE is:  511.59183910472996
Test MAE is:  1294.720658725935

Train R2 is:  0.979188471257566
Test R2 is:  0.8812560803325608


In [10]:
from sklearn. ensemble import BaggingRegressor

rf_bag = BaggingRegressor(base_estimator=RandomForestRegressor(), n_estimators=100, random_state=0)
rf_bag.fit(X_train,y_train)

print('Train MSE is: ', mean_squared_error(rf.predict(X_train), y_train))
print('Test MSE is: ', mean_squared_error(rf.predict(X_test), y_test))
print()
print('Train RMSE is: ',  math.sqrt(mean_squared_error(rf.predict(X_train), y_train)))
print('Test RMSE is: ', math.sqrt(mean_squared_error(rf.predict(X_test), y_test)))
print()
print('Train MAE is: ', mean_absolute_error(rf.predict(X_train), y_train))
print('Test MAE is: ', mean_absolute_error(rf.predict(X_test), y_test))
print()
print('Train R2 is: ', r2_score(rf.predict(X_train), y_train))
print('Test R2 is: ', r2_score(rf.predict(X_test), y_test))

Train MSE is:  505076.1077884926
Test MSE is:  3573395.149277057

Train RMSE is:  710.6870674132832
Test RMSE is:  1890.3426010321666

Train MAE is:  464.8807760869816
Test MAE is:  1246.8237670943631

Train R2 is:  0.9848466750944674
Test R2 is:  0.8896732572266641


In [24]:
from sklearn. ensemble import BaggingRegressor

rf_bag = BaggingRegressor(base_estimator=RandomForestRegressor(), n_estimators=100, random_state=0)
rf_bag.fit(X_train,y_train)

print('Train MSE is: ', mean_squared_error(rf.predict(X_train), y_train))
print('Test MSE is: ', mean_squared_error(rf.predict(X_test), y_test))
print()
print('Train RMSE is: ',  math.sqrt(mean_squared_error(rf.predict(X_train), y_train)))
print('Test RMSE is: ', math.sqrt(mean_squared_error(rf.predict(X_test), y_test)))
print()
print('Train MAE is: ', mean_absolute_error(rf.predict(X_train), y_train))
print('Test MAE is: ', mean_absolute_error(rf.predict(X_test), y_test))
print()
print('Train R2 is: ', r2_score(rf.predict(X_train), y_train))
print('Test R2 is: ', r2_score(rf.predict(X_test), y_test))

KeyboardInterrupt: 

In [26]:
from sklearn.model_selection import GridSearchCV

rf_bag = BaggingRegressor(base_estimator=RandomForestRegressor(), n_estimators=100, random_state=0)
param_grid = {"n_estimators"      : [10,50,100, 200],}

gs = GridSearchCV(rf_bag, param_grid=param_grid, scoring='r2', cv= 3, n_jobs=-1, verbose = 2)

gs = gs.fit(X_train, y_train)

print(gs.best_score_)
print(gs.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


In [ ]:
from sklearn. ensemble import BaggingRegressor

bag70 = BaggingRegressor(base_estimator=None, n_estimators=70, random_state=0)
bag70.fit(X_train,y_train)

print('Train MSE is: ', mean_squared_error(bag70.predict(X_train), y_train))
print('Test MSE is: ', mean_squared_error(bag70.predict(X_test), y_test))
print()
print('Train RMSE is: ',  math.sqrt(mean_squared_error(bag70.predict(X_train), y_train)))
print('Test RMSE is: ', math.sqrt(mean_squared_error(bag70.predict(X_test), y_test)))
print()
print('Train MAE is: ', mean_absolute_error(bag70.predict(X_train), y_train))
print('Test MAE is: ', mean_absolute_error(bag70.predict(X_test), y_test))
print()
print('Train R2 is: ', r2_score(bag70.predict(X_train), y_train))
print('Test R2 is: ', r2_score(bag70.predict(X_test), y_test))

In [21]:
from sklearn.ensemble import StackingRegressor

all_models = [rf, xgb]
 
s_train, s_test = StackingRegressor(all_models, X_train, X_test,
                           y_train)

# Train the stacking model
stacking_model = xgb
 
stacking_model = stacking_model.fit(s_train, y_train)

print('Train MSE is: ', mean_squared_error(stacking_model.predict(X_train), y_train))
print('Test MSE is: ', mean_squared_error(stacking_model.predict(X_test), y_test))
print()
print('Train RMSE is: ',  math.sqrt(mean_squared_error(stacking_model.predict(X_train), y_train)))
print('Test RMSE is: ', math.sqrt(mean_squared_error(stacking_model.predict(X_test), y_test)))
print()
print('Train MAE is: ', mean_absolute_error(stacking_model.predict(X_train), y_train))
print('Test MAE is: ', mean_absolute_error(stacking_model.predict(X_test), y_test))
print()
print('Train R2 is: ', r2_score(stacking_model.predict(X_train), y_train))
print('Test R2 is: ', r2_score(stacking_model.predict(X_test), y_test))

TypeError: __init__() takes from 2 to 3 positional arguments but 5 were given

In [22]:
from sklearn.ensemble import StackingRegressor
estimators = [('xgb', xgb),('bag', rf)]
reg = StackingRegressor(
     estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,
                                           random_state=42)
 )

reg.fit(X_train, y_train).score(X_test, y_test)

c:\Users\Jammie99\anaconda3\envs\pysnowpark\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\Jammie99\anaconda3\envs\pysnowpark\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.8842963130611153

In [23]:
print('Train MSE is: ', mean_squared_error(reg.predict(X_train), y_train))
print('Test MSE is: ', mean_squared_error(reg.predict(X_test), y_test))
print()
print('Train RMSE is: ',  math.sqrt(mean_squared_error(reg.predict(X_train), y_train)))
print('Test RMSE is: ', math.sqrt(mean_squared_error(reg.predict(X_test), y_test)))
print()
print('Train MAE is: ', mean_absolute_error(reg.predict(X_train), y_train))
print('Test MAE is: ', mean_absolute_error(reg.predict(X_test), y_test))
print()
print('Train R2 is: ', r2_score(reg.predict(X_train), y_train))
print('Test R2 is: ', r2_score(reg.predict(X_test), y_test))

Train MSE is:  3369214.133712669
Test MSE is:  4082879.1257644733

Train RMSE is:  1835.5419182662838
Test RMSE is:  2020.6135518115466

Train MAE is:  1176.4236849488614
Test MAE is:  1344.73449069789

Train R2 is:  0.8982178505755338
Test R2 is:  0.8756355272877161


In [14]:
from sklearn.ensemble import VotingRegressor
# Voting ensemble model
voting_model = VotingRegressor(estimators=[('xg', xgb), 
                                               ('rf', rf)])

# Train the voting model
voting_model.fit(X_train, y_train)

print('Train MSE is: ', mean_squared_error(voting_model.predict(X_train), y_train))
print('Test MSE is: ', mean_squared_error(voting_model.predict(X_test), y_test))
print()
print('Train RMSE is: ',  math.sqrt(mean_squared_error(voting_model.predict(X_train), y_train)))
print('Test RMSE is: ', math.sqrt(mean_squared_error(voting_model.predict(X_test), y_test)))
print()
print('Train MAE is: ', mean_absolute_error(voting_model.predict(X_train), y_train))
print('Test MAE is: ', mean_absolute_error(voting_model.predict(X_test), y_test))
print()
print('Train R2 is: ', r2_score(voting_model.predict(X_train), y_train))
print('Test R2 is: ', r2_score(voting_model.predict(X_test), y_test))

c:\Users\Jammie99\anaconda3\envs\pysnowpark\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Train MSE is:  1451568.762417508
Test MSE is:  3319054.5636782767

Train RMSE is:  1204.8106749267738
Test RMSE is:  1821.8272595606525

Train MAE is:  806.9354862980454
Test MAE is:  1205.5964445842899

Train R2 is:  0.9551208907789177
Test R2 is:  0.896611277153989
